# Computervisie

In deze week gaan we vooral focussen op een aantal technieken om te werken met visuele inputs.
Als eerste gaan we een aantal technieken zien hoe je deze data op een eenvoudige manier kan inlezen en verwerken.
Daarna gaan we een aantal veelgebruikte technieken zijn om meer data aan te maken via varianten van bestaande data.
Ten slotte ga je leren hoe je op een efficiente manier modellen kan trainen om classificatie of regressie uit te voeren op een dataset van beelden.

## Data inlezen

Vorige week heb je geleerd om gestructureerde data in te lezen uit csv-bestanden.
Om datasets voor computervisie in te lezen is het vaak complexer omdat typisch elke input in een aparte file staat (1 per beeldje).
Het label/target van de figuur kan dan in een csv-bestand staan, in de naam van het bestand of folder staan.
Hoe deze data ingeladen wordt hangt dus af van de dataset.
In deze notebook staan een aantal voorbeelden om dit te doen met pytorch en met keras.

### Pytorch

Hieronder staan de meest voorkomende voorbeelden om dit te doen met pytorch:
* Veel gebruikte klassieke datasets vind je in de torchvision package, meer bepaald onder datasets.
* Via de ImageFolder klasse in het geval dat elke klasse in een aparte subfolder aanwezig is
* Je maakt een eigen custom-dataset klasse dat bepaald hoe de figuren ingeladen worden en wat hun label is.

Hieronder staan code-voorbeelden van deze technieken

In [4]:
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
print(trainset)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

for images, labels in train_loader:
    print(images)
    break

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
tensor([[[[0.8235, 0.8196, 0.8235,  ..., 0.7725, 0.7647, 0.7647],
          [0.8196, 0.8118, 0.8157,  ..., 0.7647, 0.7608, 0.7569],
          [0.8196, 0.8078, 0.8118,  ..., 0.7647, 0.7647, 0.7569],
          ...,
          [0.9333, 0.9255, 0.7020,  ..., 0.7804, 0.7451, 0.8078],
          [0.9451, 0.9412, 0.9412,  ..., 0.9373, 0.9333, 0.9255],
          [0.9451, 0.9333, 0.9490,  ..., 0.9137, 0.9020, 0.8941]],

         [[0.8314, 0.8275, 0.8314,  ..., 0.7725, 0.7647, 0.7647],
          [0.8275, 0.8196, 0.8235,  ..., 0.7647, 0.7608, 0.7569],
          [0.8275, 0.8157, 0.8196,  ..., 0.7647, 0.7647, 0.7569],
          ...,
          [0.9922, 0.9765, 0.7373,  ..., 0.7725, 0.7412, 0.8157],
          [1.0000, 0.9882, 0.9765,  ..., 0.9608, 0.9647, 0.9686],
          [1.0000, 0.9882, 0.988

In [7]:
dataset = datasets.ImageFolder(root='dataset', transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)

for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([16, 3, 32, 32]) torch.Size([16])


In [12]:
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform):
        # bewaar de variabelen
        self.root_dir = image_dir
        self.transform = transform

        # lijst van klassen ophalen
        self.classes = sorted(os.listdir(self.root_dir))
        # golden = 1, bulldog =2, ...
        self.classes_to_idx = {cls:idx for idx, cls in enumerate(self.classes)}

        # welke afbeeldingen zijn er en wat is hun klasses
        self.image_paths = []
        self.labels = []

        for cls in self.classes:
            cls_dir = os.path.join(self.root_dir, cls)
            # voor alle images in de folder root/cls
            for img_name in os.listdir(cls_dir):
                self.image_paths.append(os.path.join(cls_dir, img_name))
                self.labels.append(self.classes_to_idx[cls])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        
        return image, label

# andere transform nodig want de afbeeldingen zijn niet even groot
transform = transforms.Compose([
    transforms.Resize((50,50)),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

dataset = CustomImageDataset(image_dir='dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([16, 3, 50, 50]) torch.Size([16])


### Keras

De technieken in Keras om dit uit te voeren zijn gelijkaardig.

Hieronder staan de meest voorkomende voorbeelden om dit te doen met pytorch:
* Veel gebruikte klassieke datasets vind je in de torchvision package, meer bepaald onder datasets.
* Via de ImageFolder klasse in het geval dat elke klasse in een aparte subfolder aanwezig is
* Je maakt een eigen custom-dataset klasse dat bepaald hoe de figuren ingeladen worden en wat hun label is.

Hieronder staan code-voorbeelden van deze technieken